# ASC Accelerometer analysis via Summary Statistics 

FDeveloping moving average acceleration and moving Standard Deviation for each data set.

Can run through a whole folder, or through random selections from the folder

## This is attempting a slicker method using matrix multiplication.  With that can then attempt GPU parallelization

In [1]:
#Standard Header used on the projects

#first the major packages used for math and graphing
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler
import scipy.special as sp
import pandas as pd

#Custome graph format style sheet
plt.style.use('Prospectus.mplstyle')

#If being run by a seperate file, use the seperate file's graph format and saving paramaeters
#otherwise set what is needed
if not 'Saving' in locals():
    Saving = False
if not 'Titles' in locals():
    Titles = True
if not 'Ledgends' in locals():
    Ledgends = True
if not 'FFormat' in locals():
    FFormat = '.png'
if not 'location' in locals():
    #save location.  First one is for running on home PC, second for running on the work laptop.  May need to make a global change
    #location = 'E:\\Documents\\Dan\\Code\\Prospectus\\Document\\Figures\\'
    #location = 'C:\\Users\\dhendrickson\\Documents\\github\\FigsAndPlots\\FigsAndPlotsDocument\\Figures\\'
    location = 'E:\\Documents\\Dan\\Phd\\Play\\'

#Standard cycle for collors and line styles
default_cycler = (cycler('color', ['0.00', '0.40', '0.60', '0.70']) + cycler(linestyle=['-', '--', ':', '-.']))
plt.rc('axes', prop_cycle=default_cycler)
my_cmap = plt.get_cmap('gray')

In [2]:
#Extra Headers:
import os as os
import statistics as st
import os as os
import pandas as pd
import random
import multiprocessing
from joblib import Parallel, delayed
import time


In [3]:
Header = np.array(['T', 'X','Y','Z','R','Theta','Phi'])

In [4]:
def getAcceleration(FileName):
    try:
        DataSet = np.genfromtxt(open(FileName,'r'), delimiter=',',skip_header=0)
        if FileName[64:69] == 'Accel':
            return [[FileName[61:],'x',DataSet[:,2]],[FileName[61:],'y',DataSet[:,3]],[FileName[61:],'z',DataSet[:,4]]]
        else:
            return [False,FileName,False]
    except:
        return [False,FileName,False]


def makeStats(DataArray):
    try:
        Arange = 50
        length = np.shape(DataArray[2])[0]
        StdDev = np.zeros(length)
        for j in range(length-Arange):
            k = (length-1)-j
            DataArray[2][k] = np.average(DataArray[2][k-Arange:k])
            StdDev[k]=st.stdev(DataArray[2][k-Arange:k])
        return [DataArray[0],DataArray[1],max(DataArray[2]),max(StdDev)]
    except:
        return ['','','','']


In [5]:
#Inputs:
folder1 = 'C:\\Users\\Hendrickson\\Desktop\\Phone Acceleration\\'
folder2 = 'E:\\Documents\\Dan\\PhD\\Data Backup\\ASC Accel Pi\\Excel Versions\\'

files = os.listdir(folder2)

Groups = 1
GroupSize = 100

RollingSize = 50

In [11]:
#Pre Calculations

num_cores = multiprocessing.cpu_count()
if np.shape(files)[0] < GroupSize:
    GroupSize = np.shape(files)[0]

if Groups !=0:
    files = random.sample(files,GroupSize*Groups-1)

loops = int(float(np.size(files))/float(GroupSize))+1
start = time.time()

inverseRollingSize = 1 / RollingSize

In [12]:
if __name__ == "__main__":
    AllEvents=[]
    Fails = []
    for i in range(loops):
        
        AllAccels = Parallel(n_jobs=num_cores)(delayed(getAcceleration)(folder2+file) for file in files[i*GroupSize:((i+1)*GroupSize)])
        Flattened = []
        DataOnlyMatrix = []
        for j in range(np.shape(AllAccels)[0]):
            if AllAccels[j][0] == False :
                if AllAccels[j][1][4:9] =='Accel':
                    print(j,AllAccels[j][1])
            else: 
                for k in range(3):
                    Flattened.append(AllAccels[j][k])
                    DataOnlyMatrix.append(AllAccels[j][k][2])
        DataOnlyMatrix = np.matrix(DataOnlyMatrix)
        
        length = np.shape(DataOnlyMatrix)[1]
        Weights = np.zeros((length,length))
        
        for j in range(length):
            r = j+1
            
            if r < RollingSize:
               ir = 1/ r
               for k in range(r):
                    Weights[j,k+1] = ir
            else:
                for k in range(RollingSize):
                    Weights[j,r-RollingSize+k] = inverseRollingSize
    

MemoryError: Unable to allocate 26.8 GiB for an array with shape (60000, 60000) and data type float64

In [9]:
np.shape(DataOnlyMatrix)

(165, 60000)

In [ ]:
        
        Events =  Parallel(n_jobs=num_cores)(delayed(makeStats)(DataArrays) for DataArrays in Flattened)
        Events = np.matrix(Events)
        if AllEvents ==[]:
            AllEvents=Events
        else:
            try:
                AllEvents=np.concatenate((AllEvents,Events), axis=0)
            except:
                Fails.append(Events)
        print(str(i+1)+' of '+str(loops),'Time: '+ str((time.time()-start)/60.0))
    
    df = pd.DataFrame(data=AllEvents)
    df.to_csv(folder1 + 'StatisticsReport.csv', sep=',', index = False, header=False,quotechar='"')

In [70]:
df = pd.DataFrame(data=AllEvents)
df.to_csv(folder1 + 'StatisticsReport.csv', sep=',', index = False, header=False,quotechar='"')